<a href="https://colab.research.google.com/github/bankuyo/demos/blob/main/Avro_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Apach Avroデモ（生成AI Geminiを使っています）
このデモでは、Apache Avroライブラリを使って、フィールドの追加と削除を行いながら後方互換性を維持する方法を紹介します。

## Avroとは
Avroは、Hadoopエコシステムで広く利用されるデータシリアライゼーションシステムです。以下のような特徴を持ちます。

- コンパクトなバイナリ形式: データを効率的に保存・転送できます。
- スキーマベース: スキーマによってデータ構造を定義し、型安全性を確保します。
- スキーマ進化: スキーマの変更に対応し、後方互換性を維持できます。
- 言語非依存: 様々なプログラミング言語で利用できます。

### スキーマ進化と後方互換性の重要性
大規模なデータ処理や分散システムにおいて、スキーマの変更は避けられない場合があります。Avroのスキーマ進化機能は、スキーマを変更しつつも、古いデータとの互換性を維持できるため、システムの安定稼働に貢献します。

### デモ概要
このデモでは、Apache Avroライブラリを使って、フィールドの追加と削除を行いながら後方互換性を維持する方法を紹介します。具体的には、以下の3つのバージョンのスキーマを定義し、データの読み書きを行います。

- バージョン1 (v1): 初期スキーマ (name, age)
- バージョン2 (v2): email フィールドを追加
- バージョン3 (v3): age フィールドを削除

## 1. 準備
まず、必要なライブラリをインストールします。

In [ ]:
!pip install avro

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for avro: filename=avro-1.11.3-py2.py3-none-any.whl size=123913 sha256=718071f372a224d8a63967a01ba7df25755e04f7f06aae5abe73f00bd2e4effe
  Stored in directory: /root/.cache/pip/wheels/1d/f6/41/0e0399396af07060e64d4e32c8bd259b48b98a4a114df31294
Successfully built avro


## 2. Avroスキーマの定義

Avroでは、スキーマを使ってデータ構造を定義します。ここでは、userというレコード（構造体のようなもの）を定義します。
ここではuser レコードの初期バージョン (v1) を定義します。

In [ ]:
import avro.schema
from avro.datafile import DataFileReader, DataFileWriter
from avro.io import DatumReader, DatumWriter

schema_v1_string = """
{
  "type": "record",
  "name": "user",
  "fields": [
    {"name": "name", "type": "string"},
    {"name": "age", "type": "int"}
  ]
}
"""

schema_v1 = avro.schema.parse(schema_v1_string)

## 3. 初期データの書き込み (v1)

初期スキーマ (v1) を使ってデータを書き込みます。



In [ ]:
records_v1 = [
    {"name": "Alice", "age": 30},
    {"name": "Bob", "age": 25},
]

with open("users_v1.avro", "wb") as out:
    writer = DataFileWriter(out, DatumWriter(), schema_v1)
    for record in records_v1:
        writer.append(record)
    writer.close()


## 4. スキーマの進化 (v2: フィールド追加)

スキーマに新しいフィールド email を追加し、バージョン2 (v2) とします。



In [ ]:
schema_v2_string = """
{
  "type": "record",
  "name": "user",
  "fields": [
    {"name": "name", "type": "string"},
    {"name": "age", "type": "int"},
    {"name": "email", "type": ["null", "string"], "default": null}
  ]
}
"""

schema_v2 = avro.schema.parse(schema_v2_string)

## 5. 新しいスキーマ (v2) でデータを読み込み

新しいスキーマ (v2) を使って、古いデータ (v1) を読み込んでみます。

In [ ]:
with open("users_v1.avro", "rb") as fo:
    reader = DataFileReader(fo, DatumReader(writers_schema=schema_v1, readers_schema=schema_v2))
    for record in reader:
        print(record)  # email フィールドはデフォルト値 null で補完される
    reader.close()


{'name': 'Alice', 'age': 30, 'email': None}
{'name': 'Bob', 'age': 25, 'email': None}


## 6. スキーマの進化 (v3: フィールド削除)

スキーマから age フィールドを削除し、バージョン3 (v3) とします。

In [ ]:
schema_v3_string = """
{
  "type": "record",
  "name": "user",
  "fields": [
    {"name": "name", "type": "string"},
    {"name": "email", "type": ["null", "string"], "default": null}
  ]
}
"""

schema_v3 = avro.schema.parse(schema_v3_string)


## 7. 新しいスキーマ (v3) でデータを読み込み

新しいスキーマ (v3) を使って、古いデータ (v1) を読み込んでみます。

In [ ]:
with open("users_v1.avro", "rb") as fo:
    reader = DataFileReader(fo, DatumReader(writers_schema=schema_v1, readers_schema=schema_v3))
    for record in reader:
        print(record)  # age フィールドは無視される
    reader.close()


{'name': 'Alice', 'email': None}
{'name': 'Bob', 'email': None}


## 解説

- 後方互換性 (フィールド追加): 新しいスキーマ (v2) で古いデータ (v1) を読む場合、追加された email フィールドにはデフォルト値 null が設定されます。
- 後方互換性 (フィールド削除): 新しいスキーマ (v3) で古いデータ (v1) を読む場合、削除された age フィールドは無視されます。
- writers_schema と readers_schema: DatumReader のコンストラクタで、書き込み時のスキーマ (writers_schema) と読み込み時のスキーマ (readers_schema) を指定することで、異なるスキーマ間の変換を適切に処理できます。